# Gemma 3  
Gemma 3 is released on 3/12/2025.

I'll test Gemma-3-4b-instruct in this notebook.  
https://huggingface.co/google/gemma-3-4b-it

In [ ]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3


## pipeline api

In [2]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get('HF_API_KEY') # replace with your own huggingface token

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3-4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16
)


In [15]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=200)
print(output[0]["generated_text"][-1]["content"])
# Okay, let's take a look!
# Based on the image, the animal on the candy is a **turtle**.
# You can see the shell shape and the head and legs.


Okay, let's take a look at the candy!

Based on the image, the animal on the candy is a **turtle**. You can see the shell shape and the head and legs. 

Do you want to know anything more about these candies?


## gradio interface|

In [ ]:
!pip install gradio

In [33]:
from transformers.image_utils import load_image
import gradio as gr
def process_image_and_generate(user_message, image):

    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."}]
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": load_image(image)},
                {"type": "text", "text": user_message}
            ]
        }
    ]

    output = pipe(text=messages, max_new_tokens=1000)
    response = output[0]["generated_text"][-1]["content"]
    return response


with gr.Blocks() as demo:
    gr.Markdown("# Gemma-3-4b-it chat")
    image_input = gr.Image(type="pil", label="Upload an image")
    user_message = gr.Textbox(label="User Prompt")
    with gr.Accordion('Answer:', open=True):
        image_output = gr.Markdown(label="Output",)
    image_button = gr.Button("Process Image")

    image_button.click(
        fn=process_image_and_generate,
        inputs=[user_message, image_input],
        outputs=image_output,
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://971706fbd0f4b36c2d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
